## Lista 1 - Lucas Carneiro

Organizando os dados

In [44]:
import os

import nltk
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import PortugueseStemmer

import string

#import enchant
#PyEnchant não possui suporte para Windows x64

from collections import defaultdict

In [3]:
nltk.download('stopwords')
nltk.download('machado')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Casa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package machado to
[nltk_data]     C:\Users\Casa\AppData\Roaming\nltk_data...
[nltk_data]   Package machado is already up-to-date!


True

In [10]:
textos = []
for i in machado.fileids():
    textos.append(machado.raw(i))

Organizando o texto

In [12]:
swu = stopwords.words('portuguese') + list (string.punctuation)
stemmer = PortugueseStemmer()

In [13]:
textos_limpos = []
for texto in textos:
    tlimpo = [token.lower() for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_limpos.append(tlimpo)

In [16]:
textos_limpos2 = []
for texto in textos_limpos:
    tlimpo2 = [stemmer.stem(token) for token in texto]
    textos_limpos2.append(tlimpo2)

In [19]:
print(textos_limpos[0][0:10])
print(textos_limpos2[0][0:10])

['conto', 'contos', 'fluminenses', '1870', 'contos', 'fluminenses', 'texto', 'fonte', 'obra', 'completa']
['cont', 'cont', 'fluminens', '1870', 'cont', 'fluminens', 'text', 'font', 'obra', 'complet']


Gerando indices invertidos

In [20]:
indice = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_limpos):
    for term in t:
        indice[term].add(tid)

In [21]:
indice2 = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_limpos2):
    for term in t:
        indice2[term].add(tid)

Função de busca

In [24]:
def busca(search, indice):
    
    #operadores disponiveis:
    ##'and'
    ##'or'
    ##'not'  
    
    #os operadores são executados em sequência
    #o operador segue funcional até outro operador ser sugerido
    #o operador padrão é o 'or'
    
    tokens = WordPunctTokenizer().tokenize(search)

    resultado = set()
    operador = 'or'
    
    for token in tokens:
        if token == 'and':
            operador = 'and'
            
        elif token == 'or':
            operador = 'or'
        
        elif token == 'not':
            operador = 'not'
            
        else:
            resultado_temp = indice[token]
            if operador == 'and':
                resultado = resultado & resultado_temp
            
            elif operador == 'not':
                resultado = resultado - resultado_temp
                
            else:
                resultado = resultado | resultado_temp
    
    return resultado

### Exercício 1

O conjunto relevante considerado é a busca de termos stemizados no índice stemizado.
A função a seguir retorna a Revocação e a Precisão de uma busca

In [41]:
def RevPre(parcial,geral):
    
    #parcial = 'documentos encontrados'
    #geral = 'documentos relevantes'
    
    rev = len(parcial & geral)/len(geral)
    pre = len(parcial & geral)/len(parcial)
    
    print('Revocação = ' + str(rev))
    print('Precisão = ' + str(pre))
    
    return rev,pre

In [39]:
palavra = 'ator'

resultado = busca(stemmer.stem(palavra),indice2)
resultado1 = busca(palavra,indice)

RevPre(resultado1,resultado)

Revocação = 0.65625
Precisão = 1.0


(0.65625, 1.0)

### Exercício 2

In [40]:
equi = ['ator', 'atores','atriz','atrizes']

resultado2 = set()
for palavra in equi:
    resultado2 = resultado2 | busca(palavra,indice)
    
RevPre(resultado2,resultado)

Revocação = 1.0
Precisão = 0.9142857142857143


(1.0, 0.9142857142857143)

Houve uma grande melhora na revocação ao custo de uma pequena precisão

### Exercício 3

Redefinindo a função de busca

In [ ]:
def busca_corr_sup(termo,ind,dic):

    #Função suporte de busca_corr
    #faz a verificação ortográfica e a busca sobre os termos possíveis
    
    if not corretor.check(termo):
        resultado = set()
        for token in dic.suggest(termo):
            resulado = resultado | ind[token]
    else:
        resultado = ind[termo]
        
    return resultado
    
def busca_corr(search, indice):
    
    #operadores disponiveis:
    ##'and'
    ##'or'
    ##'not'  
    
    #os operadores são executados em sequência
    #o operador segue funcional até outro operador ser sugerido
    #o operador padrão é o 'or'
    
    corretor = enchant.Dict("pt_BR")
    tokens = WordPunctTokenizer().tokenize(search)

    resultado = set()
    operador = 'or'
    
    for token in tokens:
        if token == 'and':
            operador = 'and'
            
        elif token == 'or':
            operador = 'or'
        
        elif token == 'not':
            operador = 'not'
            
        else:
            
            #caso um termo esteja escrito errado, faz-se a busca sobre todas as correções sugeridas
            
            resultado_temp = busca_corr_sup(token,indice,corretor)
               
            if operador == 'and':
                resultado = resultado & resultado_temp
            
            elif operador == 'not':
                resultado = resultado - resultado_temp
                
            else:
                resultado = resultado | resultado_temp
    
    return resultado

### Exercício 4

Criando novo indice para consulta por frases

In [52]:
indice_f = defaultdict(lambda: defaultdict(list))

for texto_id,texto in enumerate(textos_limpos2):
    for termo_id,termo in enumerate(texto):
        indice_f[termo][texto_id].append(termo_id)

Criando nova função de busca

In [102]:
def prox_pos(lista1,lista2,raio):
    
    #Compara a proximidade entre termos de duas listas e devolve a média dos valores quando proximos
    #Por que a média? Para a captar termos que estejam pertos, pelo menos, de boa parte do conjunto de tokens próximos
    #Aumentando a revocação em sacrifício da precisão
    
    resultado = []
    
    for i in lista1:
        for j in lista2:
            if abs(i-j) <= raio:
                resultado.append(round((i+j)/2))
                
    return resultado

def busca_f(frase, indice, raio = 1):
    
    #frase = frase a ser buscada
    #indice = indice invertido
    #raio = distancia maxima entre os termos
    
    tokens = [stemmer.stem(token.lower())  for token in WordPunctTokenizer().tokenize(frase) if token not in swu]

    #Verificar quais textos possuem todos os termos
    textos_comuns = indice[tokens[0]].keys()
    for token in tokens[1:]:
        textos_comuns = textos_comuns & indice[token].keys()
    
    #Comparar as posições dentro dos textos em comum
    resultado = defaultdict(list)
    for texto in textos_comuns:
        
        resultado_temp = indice[tokens[0]][texto]
        for token in tokens[1:]:
            resultado_temp = prox_pos(resultado_temp,indice[token][texto],raio)
            
        if resultado_temp:
            resultado[texto] = resultado_temp
            
    return resultado

Testando a nova função busca

In [106]:
busca_f('Dom Casmurro',indice_f)

defaultdict(list, {230: [2, 4, 112, 136, 144, 164, 17460, 17462, 38852]})

Obs: provavelmente o texto 230 é a obra "Dom Casmurro"

### Exercício 5

A ideia é criar a nova função de busca reduzindo os termos procurados, dando prioridade aos termos que estiverem na frente da frase procurada

In [100]:
def busca_f_adv(frase, indice, raio = 1):
    
    #frase = frase a ser buscada
    #indice = indice invertido
    #raio = distancia maxima entre os termos
    
    tokens = [stemmer.stem(token.lower())  for token in WordPunctTokenizer().tokenize(frase) if token not in swu]

    #Verificar quais textos possuem todos os termos
    textos_comuns = indice[tokens[0]].keys()
    for token in tokens[1:]:
        textos_comuns = textos_comuns & indice[token].keys()

    #Ainda que a busca seja reduzida os termos são procurados nos textos que possuem todas os itens buscados
    #Por que?  para que uma busca específica não seja transformada em outra distante do objetivo original
    while tokens:
        
        #Comparar as posições dentro dos textos em comum
        resultado = defaultdict(list)
        for texto in textos_comuns:

            resultado_temp = indice[tokens[0]][texto]
            for token in tokens[1:]:
                resultado_temp = prox_pos(resultado_temp,indice[token][texto],raio)

            if resultado_temp:
                resultado[texto] = resultado_temp

        if not resultado:
            tokens = tokens[:-1]
        else:  
            return resultado

In [107]:
busca_f_adv('Dom conto rua',indice_f)

defaultdict(list,
            {0: [21159, 28210],
             1: [1973],
             2: [467, 5288, 12274, 23175],
             3: [10668, 21399],
             5: [11155],
             12: [9069],
             24: [499, 520, 1971],
             33: [1031, 1082, 1150],
             34: [242],
             39: [9774],
             41: [2344],
             43: [1500],
             49: [1255],
             51: [2024],
             52: [491],
             62: [4043],
             71: [4522],
             94: [519],
             140: [8166],
             175: [178, 1022],
             182: [2452],
             185: [21064, 26008, 30803],
             186: [667, 1634],
             188: [3468, 6056, 6178],
             191: [1397,
              14432,
              14462,
              14483,
              14503,
              14531,
              14548,
              14580,
              14597,
              14646,
              14668,
              14722,
              14838,
            